<a href="https://colab.research.google.com/github/YKannikanti/Yasasri-/blob/main/week11/week11_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `/data/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [1]:
import pandas as pd

def load_and_filter_data (file_path):
    df = pd.read_csv(file_path)
    mo_hospitals = df[df['State'] == 'MO']
    start_date_df= pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')
    mo_hospitals['Start Date'] = start_date_df['Start Date']
    return mo_hospitals

    file_path = '/data/complications_all.csv'
    mo_hospitals = load_and_filter_data(file_path)

    mo_hospitals.head()



In [2]:
import pandas as pd

def clean_data(df):
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
    df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')
    #remove rows where 'denominator' is not available
    df = df[df['Denominator'] != 'Not Available']
    #convert 'denominator'to numeric
    df['Denominator'] = pd.to_numeric(df['Denominator'],errors='coerce')
    return df

file_path = 'complications_all.csv'

mo_hospitals = load_and_filter_data(file_path)
mo_hospitals.head()


<ipython-input-1-e4c3dff5c048>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['Start Date'] = start_date_df['Start Date']


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
45534,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,26,2.5,1.4,4.2,NaN,04/01/2015,03/31/2018
45535,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,175,13.9,11.0,16.9,NaN,07/01/2015,06/30/2018
45536,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,91,2.5,1.2,5.1,NaN,07/01/2015,06/30/2018
45537,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,326,8.5,6.5,10.9,NaN,07/01/2015,06/30/2018
45538,260001,MERCY HOSPITAL JOPLIN,100 MERCY WAY,JOPLIN,MO,64804,JASPER,(417) 781-2727,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,461,13.1,10.7,15.9,NaN,07/01/2015,06/30/2018


In [18]:
import pandas as pd

file_path = 'complications_all.csv'

mo_hospitals = load_and_filter_data(file_path)

mo_hospitals= clean_data(mo_hospitals)

mo_summary = mo_hospitals.groupby('Facility Name').agg({
    'Start Date': 'min',
    'End Date': 'max',
    'Denominator': 'sum'
})

mo_summary.index.name ='Hospital Name'


mo_summary = mo_summary.rename(columns={
    'Start Date': 'start_date',
    'End Date': 'end_date',
    'Denominator': 'number'
})
actual_min_start_date = mo_summary['start_date'].min()
print(f"actual minimum start date: {mo_summary['start_date'].min()}")

assert isinstance(actual_min_start_date, pd.Timestamp)
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

mo_summary.head()

actual minimum start date: 2015-04-01 00:00:00


<ipython-input-1-e4c3dff5c048>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['Start Date'] = start_date_df['Start Date']
<ipython-input-2-5c1ac509863d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Denominator'] = pd.to_numeric(df['Denominator'],errors='coerce')


,start_date,end_date,number
Hospital Name,,,
BARNES JEWISH HOSPITAL,2015-04-01,2018-06-30,131313
BARNES-JEWISH ST PETERS HOSPITAL,2015-04-01,2018-06-30,15668
BARNES-JEWISH WEST COUNTY HOSPITAL,2015-04-01,2018-06-30,9622
BATES COUNTY MEMORIAL HOSPITAL,2015-07-01,2018-06-30,3117
BELTON REGIONAL MEDICAL CENTER,2015-04-01,2018-06-30,9270


---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

For my project,I'm planning to use three different sources of data to keep things versatile and to show how different types of data access work together in a real-world application.

AWS S3:I'll use this to store large datasets that I want to pull in and analyze,especially if they're too big to keep locally.AWS S3 is  great because it's scalable and easily accessible,which is practical for any larger data analysis work.

Relational Database (PostgreSQL):I'll connect to a postgreSQL database aato manage and query structured data.Relational databases are powerful for handling data relationships and complex queries,which is useful in projects that need consistent,structured data access.

Web API :I plan to use an external REST API,which will let me bring in dynamic,up-to-date data.For instance,I might pull in real-time weather or financial information to show how external data can enrich and complement other sources.

#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

To keep things flexible and demonstrate handling of different formats,I'll use:

CSV:It's a common format  for structured,tabular data and easy to work with using python's pandas library.

JSON: APIs often provide data in JSON,which is structured and good for handling nested data,like a hierarchy of information about each entry.

EXCEL: Some data might come in an Excel file format,which is practical for pre-formated datasets that could come from a business reports.It's also something many stakeholders in real life would be familiar with.


#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

The goal of my project is to create a tool that can integrate data from various sources into one streamlined analysis platform.This could be super useful in industries like healthcare or finance,where data from electronic health records,external health monitoring devices,and maybe even government data sources.Combining all of these can give administrators  a much clearer picture.

This project could ultimately help users make better,data-driven decisions.In a real work setting,having an all-in-one analysis platform saves time,reduces errors,and allows for a more comprehensive view of whatever data you're working with.I think it's an interesting and practical idea that could genuinely improve data management and decisison-making in any data-intensive field.




---



## Submit your work via GitHub as normal
